In [14]:
# 1️⃣ تثبيت المكتبات
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q torchaudio pydub rapidfuzz gTTS

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.7 MB/s eta 0:00:00


In [15]:
# 2️⃣ استيراد المكتبات
import whisper
import os
from pydub import AudioSegment
from IPython.display import display, Javascript, Audio, HTML
from google.colab import output
import base64
import json
from rapidfuzz import fuzz
from gtts import gTTS

In [16]:

# 3️⃣ رفع ملف stories.json
from google.colab import files
print("ارفع ملف stories.json:")
uploaded_json = files.upload()
json_path = list(uploaded_json.keys())[0]

ارفع ملف stories.json:


Saving stories.json to stories.json


In [17]:

# 4️⃣ تحميل القصص
with open(json_path, "r", encoding="utf-8") as f:
    stories = json.load(f)

print(f"✅ تم تحميل {len(stories)} قصة/عنصر محتوى")


✅ تم تحميل 25 قصة/عنصر محتوى


In [18]:
# 5️⃣ تحميل نموذج Whisper
model = whisper.load_model("small")


In [19]:

# 6️⃣ دالة لتحويل webm إلى wav
def convert_webm_to_wav(webm_file, wav_file):
    audio = AudioSegment.from_file(webm_file, format="webm")
    audio.export(wav_file, format="wav")


In [30]:
from rapidfuzz import fuzz

def search_stories_partial(query):
    query = query.lower().strip()
    results = []

    for s in stories:
        story_text = (s.get("content") or s.get("text") or "").lower()
        story_title = s.get("title", "").lower()

        # حساب التشابه
        score_title = fuzz.partial_ratio(query, story_title)
        score_text = fuzz.partial_ratio(query, story_text)

        # الأولوية للعنوان
        if score_title > 70:
            results.append((score_title + 20, s))  # نرفع درجته عشان يظهر في الأول
        elif score_text > 70:
            results.append((score_text, s))

    # ترتيب النتائج حسب أعلى نسبة تطابق
    results.sort(reverse=True, key=lambda x: x[0])
    return [r[1] for r in results]


In [31]:
def show_story(story):
    title = story.get("title", "بدون عنوان")
    text = story.get("content") or story.get("text") or "لا يوجد نص"

    print(f"\n📖 {title}\n")
    print(text)

    # زر سماع القصة
    tts = gTTS(text, lang="ar")
    audio_file = "story_audio.mp3"
    tts.save(audio_file)
    display(Audio(audio_file, autoplay=False))


In [32]:
# 9️⃣ دالة استقبال الصوت
def receive_audio(base64_string):
    audio_bytes = base64.b64decode(base64_string)
    webm_path = "recorded_audio.webm"
    with open(webm_path, "wb") as f:
        f.write(audio_bytes)
    wav_path = "recorded_audio.wav"
    convert_webm_to_wav(webm_path, wav_path)
    print(f"✅ تم حفظ الصوت كـ {wav_path}")

    result = model.transcribe(wav_path, language="ar")
    text = result["text"]
    print(f"\n📢 نص التعرف الصوتي: {text}\n")

    results = search_stories_partial(text)
    if results:
        print("✅ اختر من بين القصص القريبة:\n")
        for idx, r in enumerate(results, 1):
            title = r.get("title", "بدون عنوان")
            # نعرض زر لكل قصة
            display(HTML(f"""
                <button onclick="google.colab.kernel.invokeFunction('notebook.show_story', [{idx-1}], {{}})"
                        style="margin:5px;padding:10px 20px;font-size:18px;background:#2196F3;color:white;border:none;border-radius:8px;">
                    {title}
                </button>
            """))
        # نخزن النتائج عشان نرجع لها لما يختار
        global last_results
        last_results = results
    else:
        print("❌ لم يتم العثور على قصة مطابقة.")

In [33]:
# 🔟 تسجيل الدوال
output.register_callback('notebook.receive_audio', receive_audio)
output.register_callback('notebook.show_story', lambda idx: show_story(last_results[idx]))


In [36]:
# 1️⃣1️⃣ زر للتسجيل
display(Javascript("""
function recordAudio() {
  navigator.mediaDevices.getUserMedia({audio:true}).then(stream => {
    const mediaRecorder = new MediaRecorder(stream);
    const audioChunks = [];
    mediaRecorder.ondataavailable = e => audioChunks.push(e.data);
    mediaRecorder.start();
    alert('🎤 سجل صوتك لمدة 5 ثوانٍ بعد الضغط على OK');
    setTimeout(() => { mediaRecorder.stop(); }, 5000);
    mediaRecorder.onstop = async () => {
      const audioBlob = new Blob(audioChunks);
      const arrayBuffer = await audioBlob.arrayBuffer();
      const base64String = btoa(String.fromCharCode(...new Uint8Array(arrayBuffer)));
      google.colab.kernel.invokeFunction('notebook.receive_audio', [base64String], {});
    };
  });
}

var button = document.createElement('button');
button.innerHTML = "🎙️ اضغط للتسجيل";
button.style.fontSize = "20px";
button.style.padding = "10px 20px";
button.style.background = "#4CAF50";
button.style.color = "white";
button.style.border = "none";
button.style.borderRadius = "10px";
button.onclick = recordAudio;
document.body.appendChild(button);
"""))

✅ تم حفظ الصوت كـ recorded_audio.wav

📢 نص التعرف الصوتي:  بزبحنا عظيم

✅ اختر من بين القصص القريبة:



✅ تم حفظ الصوت كـ recorded_audio.wav

📢 نص التعرف الصوتي:  بزبحة نالزين

❌ لم يتم العثور على قصة مطابقة.


<IPython.core.display.Javascript object>


📖 قصة إسماعيل عليه السلام

إسماعيل عليه السلام ابن إبراهيم، نشأ في مكة مع أمه هاجر. ساعد والده في بناء الكعبة. كان صادق الوعد وحليمًا. ابتلاه الله بأمر الذبح فاستسلم مع أبيه، ففداه الله بذبح عظيم.



📖 قصة إبراهيم عليه السلام

إبراهيم عليه السلام هو خليل الله وأبو الأنبياء. جادل قومه في عبادة الأصنام، وحطمها ليبين بطلانها. ألقوه في النار فجعلها الله بردًا وسلامًا. ابتلاه الله بذبح ابنه إسماعيل فاستجاب لأمر الله، ففداه الله بذبح عظيم.
